# 개요
- 텐서플로에서 데이터 플로우 그래프를 시각적으로 보여주는 텐서보드를 활용
- 텐서보드
  - 코랩용 코드, 일반 PC(아나콘다, python)에서 띠는 방식이 조금 다르다.
- 작성한 신경망을 설명할 때 자료로 사용 가능


# 텐서플로 모듈 가져오기

In [6]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

'1.15.2'

# 텐서보드 설정

In [7]:
from tensorboardcolab import *
import shutil, os

In [8]:
# 그래프 정보 저장을 위한 위치 지정
try :
  shutil.rmtree('./Graph', ignore_errors=True)
  os.mkdir('./Graph')
except Exception as e : 
  print(e)

In [9]:
# 코랩에 사용하는 텐서보드
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://8dee2dc7b269.ngrok.io


# 훈련 코드

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist     = input_data.read_data_sets('./data/mnist', one_hot=True)
PIXEL     = mnist.train.images.shape[1]
PIXEL_H   = int(np.sqrt(PIXEL))
PIXEL_W   = PIXEL_H 
LABEL_NUM = mnist.train.labels.shape[-1]

x = tf.placeholder(tf.float32, shape=(None, PIXEL), name='x')

def make_WeightFilter(shape, name) :
  tmp = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial_value=tmp, name="W_"+name)
def make_Bais(shape, value, name) :
  tmp = tf.constant(value, shape=[shape])
  return tf.Variable(tmp, name="b_"+name)
def make_Conv_2d(x, W, name) :
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME', name='conv_'+name)
def make_MaxPooling(x) : 
  return tf.nn.max_pool(value=x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
def make_FeedDictParam(featureDatas, labels, prob) :
  return {
            x         : featureDatas,
            keep_prob : prob,
            y_        : labels
          }

layer_name = 'conv_1f'
with tf.name_scope(layer_name) as scope :
  conv1_W   = make_WeightFilter([5,5,1,32], layer_name)
  conv1_b   = make_Bais(32, 0.1, layer_name)
  x_4d      = tf.reshape( x, (-1, PIXEL_H, PIXEL_W, 1) )
  conv1     = make_Conv_2d(x_4d, conv1_W, layer_name) + conv1_b
  act_conv1 = tf.nn.relu(conv1)

layer_name = "pooling_1f"
with tf.name_scope(layer_name) as scope :
  pool1 = make_MaxPooling(act_conv1)

layer_name = 'conv_2f'
with tf.name_scope(layer_name) as scope :
  conv2_W   = make_WeightFilter([5,5,32,32*2], layer_name)
  conv2_b   = make_Bais(32*2, 0.1, layer_name)
  conv2     = make_Conv_2d(pool1, conv2_W, layer_name) + conv2_b
  act_conv2 = tf.nn.relu(conv2)

layer_name = "pooling_2f"
with tf.name_scope(layer_name) as scope :
  pool2 = make_MaxPooling(act_conv2)

layer_name = "fc"
with tf.name_scope(layer_name) as scope :
  in_channels = 7 * 7 * 64
  fc_W        = make_WeightFilter([in_channels, 1024], layer_name)
  fc_b        = make_Bais( 1024, 0.1, layer_name )
  tmp_x       = tf.reshape(pool2, [-1, in_channels])
  fc          = tf.matmul(tmp_x, fc_W) + fc_b
  act_fc      = tf.nn.relu(fc)

layer_name = "fc_dropout"
with tf.name_scope(layer_name) as scope :
  keep_prob  = tf.placeholder(tf.float32)
  fc_dropout = tf.nn.dropout(act_fc, rate=1-keep_prob)

layer_name = "output"
with tf.name_scope(layer_name) as scope :
  y_W    = make_WeightFilter([int(fc_dropout.shape[-1]), LABEL_NUM], layer_name)
  y_b    = make_Bais(LABEL_NUM, 0.1, layer_name)
  y_conv = tf.matmul(fc_dropout, y_W) + y_b
  y_conv = tf.nn.softmax(y_conv)

y_ = tf.placeholder(tf.float32, shape=(None, LABEL_NUM), name='y_')

layer_name = "loss"
with tf.name_scope(layer_name) as scope :
  cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))

layer_name = "adam"
with tf.name_scope(layer_name) as scope :
  optimizer = tf.train.AdamOptimizer()
  train     = optimizer.minimize(cross_entropy)

layer_name = "predict"
with tf.name_scope(layer_name) as scope :
  predict  = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [13]:
with tf.device('/device:GPU:0') :
  with tf.Session() as sess :
    TRAIN_COUNT = 3000 
    BATCH_SIZE = 50
    VERBOSE_TERM = 100
    sess.run(tf.global_variables_initializer())
    test_fd = make_FeedDictParam(mnist.test.images, mnist.test.labels, 1.0)
    for step in range(TRAIN_COUNT) :
      batch = mnist.train.next_batch(BATCH_SIZE)
      train_fd = make_FeedDictParam(batch[0], batch[1], 0.5)
      _, loss = sess.run( [train, cross_entropy], feed_dict=train_fd )
      if step % VERBOSE_TERM == 0 :
        acc = sess.run(accuracy, feed_dict=test_fd)
        print(f'step={step:<4} acc={acc:<20}  loss={loss:<20}')
    acc = sess.run(accuracy, feed_dict=test_fd)
    print(f'step={step:<4} acc={acc:<20}  loss={loss:<20}')

    # 텐서보드 기록 작업
    writer = tbc.get_writer()
    writer.add_graph(sess.graph)
    writer.flush()

tbc.close()

step=0    acc=0.1467999964952469    loss=592.4326171875      
step=100  acc=0.9480000138282776    loss=12.890593528747559  
step=200  acc=0.9607999920845032    loss=5.807723045349121   
step=300  acc=0.9668999910354614    loss=9.498489379882812   
step=400  acc=0.9682999849319458    loss=6.469717502593994   
step=500  acc=0.9750999808311462    loss=9.37108325958252    
step=600  acc=0.9786999821662903    loss=7.847933769226074   
step=700  acc=0.9785000085830688    loss=8.517621994018555   
step=800  acc=0.9839000105857849    loss=7.538034439086914   
step=900  acc=0.9846000075340271    loss=2.8269155025482178  
step=1000 acc=0.98089998960495      loss=0.388607919216156   
step=1100 acc=0.9855999946594238    loss=1.3038325309753418  
step=1200 acc=0.982699990272522     loss=3.6846439838409424  
step=1300 acc=0.9866999983787537    loss=0.8722720146179199  
step=1400 acc=0.9836999773979187    loss=0.9423725008964539  
step=1500 acc=0.9854999780654907    loss=0.9381278157234192  
step=160